#### Imports and Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks  import EarlyStopping
from keras.models import Sequential
from keras.layers import Lambda, SimpleRNN, Dense

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ts_utils

mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

In [ ]:
# Read the data
df, df_scaled_trn, df_scaled_tst, scaler = ts_utils.load_file()
print("Training Data:")
display(df_scaled_trn)

input_slice  = slice(0,3)
label_slice  = slice(1,3)
window_len   = 3
ouput_len    = 3
batch_size   = 5

ouput_feat_len  = label_slice.stop - (label_slice.start or 0)
model_op_len    = ouput_feat_len * ouput_len

ds_trn = tf.data.Dataset.from_tensor_slices(df_scaled_trn[df_scaled_trn.columns[input_slice]])
ds_tst = tf.data.Dataset.from_tensor_slices(df_scaled_tst[df_scaled_trn.columns[input_slice]])
window_trn = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=batch_size, skip=1)
window_tst = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=batch_size, skip=1)

#print("\n\nSample Window to verify the window is working:\n")
#for w in window_trn.take(3):
#    print(f"{w[0].numpy().shape}\n{w[0].numpy()}\n=>:{w[1].numpy().shape}\n{w[1].numpy()} \n")


In [ ]:
# yh: [batch, time, features length]
def inv_transform(yh, scaler, label_slice):
    yy=np.zeros([yh.shape[0], yh.shape[1], scaler.n_features_in_])
    yy[:, :, label_slice] = yh[0]
    ys = []
    for i in range(yh.shape[0]):
        yi = scaler.inverse_transform(yy[i])
        ys.append(yi[:, label_slice])
    return np.stack(ys)
    
#inv_transform(yh, scaler, label_slice)

def predict(model, window_trn, n=1):
    for w in window_trn.take(n):
        x = w[0]
        yh = model.predict(x)
        print(x.shape, x, "\n\n", yh, yh.shape)


In [ ]:
def compile_fit(model, window_trn, window_tst= None, opt=None, patience=3, epochs=1):
    early_stop = EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    loss = tf.keras.losses.MeanSquaredError()
    opt  = opt or tf.keras.optimizers.Adam()
    mets = [tf.keras.metrics.MeanAbsoluteError()]

    ##=> Other options you can try
    #learning_rate = 1e-6
    #opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    #opt = tf.keras.optimizers.SGD()
    #loss=tf.keras.losses.Huber()

    model.compile(loss= loss, optimizer= opt, metrics=mets)

    history = []
    if (window_trn is not None):
        history = model.fit(window_trn, epochs=epochs, validation_data=window_tst, callbacks=[early_stop])

    return history

# This commonLayer, a layer that is common to all models

def getCommonLayer(ouput_len, ouput_feat_len):
    op_len = ouput_len * ouput_feat_len;
    commonLayer = [
        # Shape => [batch, 1, out_len * #features]
        tf.keras.layers.Dense( op_len, kernel_initializer=tf.initializers.zeros()),
        # Shape => [batch, out_steps, features]
        tf.keras.layers.Reshape([ouput_len, ouput_feat_len])
    ]
    return commonLayer

performance = {}
models = []

In [ ]:

srnn_model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(40)]
    + getCommonLayer(ouput_len, ouput_feat_len)
    , name="SimpleRNN")

models.append(srnn_model)
model = srnn_model
for opt in "sgd adam".split():
    history = compile_fit(model, window_trn, window_tst, epochs=5, opt=opt )
    performance[model.name + f":{opt}"] = ts_utils.eval_performance(model, window_trn, window_tst)


In [ ]:
# Linear Model
linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :])
    ] + getCommonLayer(ouput_len, ouput_feat_len),
    name = "Linear"
)
models.append(linear_model)

In [ ]:
# Add a Dense Layer

dlinear_model = tf.keras.Sequential([
        # Take the last time-step.
        # Shape [batch, time, features] => [batch, 1, features]
        tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
        tf.keras.layers.Dense(512, activation='relu')
    ] +  getCommonLayer(ouput_len, ouput_feat_len),
    name = "DenseLinear"
)

models.append(dlinear_model)

In [ ]:
# CNN
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    ] + getCommonLayer(ouput_len, ouput_feat_len),
    name = "Conv"
)
models.append(conv_model)

In [ ]:
# LSTM

lstm_model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, lstm_units].
        # Adding more `lstm_units` just overfits more quickly.

        tf.keras.layers.LSTM(32, return_sequences=False)
    ]+ getCommonLayer(ouput_len, ouput_feat_len),
    name = "LSTM"
)
models.append(lstm_model)


In [ ]:
for model in models:
    history = compile_fit(model, window_trn, window_tst, epochs=5 )
    performance[model.name] = ts_utils.eval_performance(model, window_trn, window_tst)

# Plot graphs
performance = ts_utils.plot_performance([], window_trn, window_tst, performance=performance)

In [ ]:
for w in window_trn:
    d, y = w
    yh = linear_model.predict(d)
    #print( "=>\n\n", l[0], l[1], d.shape, l.shape)
    break;

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

plt.figure(figsize=(12, 8))
max_n = 3
skip  = 1

for i in range(max_n):
    di, yi, yhi = d[i], y[i], yh[i]
    #plt.subplot(max_n, 1, i+1)
    #plt.ylim (-3,3)

    #mm = MinMaxScaler()
    #di = mm.fit_transform(di)

    for j in range(di.shape[1]):
        pass
        plt.plot(range(di.shape[0]), di[:, j], marker='.', label = "Inputs")

    for j in range(yi.shape[1]):
        xx = range( skip+di.shape[0], skip+di.shape[0]+yi.shape[1]+1)
        yy = yi[:,j]
        hy = yhi[:,j]
        plt.scatter( xx, yy, edgecolors='k', marker='o', label='Labels', c='#2ca02c', s=64)
        plt.scatter( xx, hy, edgecolors='k', marker='X', label='Labels', c='#ff7f0e', s=64)

    break;

plt.legend()